<a href="https://colab.research.google.com/github/fernando2393/DT2119-Final-Project/blob/master/MusicGenreClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
from google.colab import files
files.upload()

Saving constants.py to constants.py
Saving loader.py to loader.py
Saving main.py to main.py
Saving mfcc.py to mfcc.py


{'constants.py': b'DATASETS = "/content/drive/My Drive/Datasets/fma_small/"\r\nMETADATA = "/content/drive/My Drive/Datasets/fma_metadata/"\r\nMFCC = "/content/drive/My Drive/Datasets/mfcc.csv"\r\n',
 'loader.py': b'"""\r\nThis class is in charge of loading the relevant data and splitting the dataset.\r\nnote: Check https://nbviewer.jupyter.org/github/mdeff/fma/blob/outputs/usage.ipynb for a deeper understanding of\r\n      the data format in the .csv files.\r\n"""\r\n\r\nimport pandas as pd\r\nimport mfcc\r\nimport constants as cts\r\nfrom pathlib import Path\r\n\r\n\r\nclass Loader:\r\n    def __init__(self):\r\n        self.features = [\'mfcc\', \'chroma_cens\', \'tonnetz\', \'spectral_contrast\',\r\n                         [\'spectral_centroid\', \'spectral_bandwidth\', \'spectral_rolloff\'],\r\n                         [\'rmse\', \'zcr\']]  # Main categories of the stored features\r\n\r\n    @staticmethod\r\n    def load_features():\r\n        """\r\n        This method loads the 

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, ZeroPadding2D, ELU, Reshape, GRU
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import loader

In [4]:
(x_train_prelim, y_train_prelim), (x_val_prelim, y_val_prelim), (x_test_prelim, y_test_prelim) = loader.get_train_val_test()
print("There are the following classes:")
classes = set(y_train_prelim.tolist()) & set(y_val_prelim.tolist()) & set(y_test_prelim.tolist())
print(classes)

There are the following classes:
{'Experimental', 'Rock', 'International', 'Folk', 'Electronic', 'Instrumental', 'Pop', 'Hip-Hop'}


In [0]:
# remove corrupted samples
x_train_prelim = np.delete(x_train_prelim, 3495)
x_train_prelim = np.delete(x_train_prelim, 3495)
x_train_prelim = np.delete(x_train_prelim, 3495)
y_train_prelim = np.delete(y_train_prelim, 3495)
y_train_prelim = np.delete(y_train_prelim, 3495)
y_train_prelim = np.delete(y_train_prelim, 3495)

In [0]:
x_train = np.zeros([6397 - 3, 13, 2455])
x_val = np.zeros([800, 13, 2455])
x_test = np.zeros([800, 13, 2455])

y_train = np.zeros([6397 - 3])
y_val = np.zeros([800])
y_test = np.zeros([800])

In [0]:
for i in range(6397 - 3):
  x_train[i] = x_train_prelim[i]

for i in range(800):
  x_val[i] = x_val_prelim[i]
  x_test[i] = x_test_prelim[i]


In [0]:
x_train = x_train.reshape((6394, 13, 2455, 1))
x_val = x_val.reshape((800, 13, 2455, 1))
x_test = x_test.reshape((800, 13, 2455, 1))

In [0]:
dict_labels = {'Electronic': 0, 'Experimental': 1, 'Folk': 2, 'Hip-Hop': 3,
               'Instrumental': 4, 'International': 5, 'Pop': 6, 'Rock': 7}

for i in range(len(y_train)):
  y_train[i] = dict_labels[y_train_prelim[i]]

In [0]:
for i in range(len(y_val_prelim)):
  y_val[i] = dict_labels[y_val_prelim[i]]
  y_test[i] = dict_labels[y_test_prelim[i]]

# One-hot encoding of classes
y_train = to_categorical(y_train, num_classes=8)
y_val = to_categorical(y_val, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)

In [12]:
x_train.shape

(6394, 13, 2455, 1)

In [13]:
y_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [0]:
class Classifier:
    @staticmethod
    def build(width, height, depth, classes):
        
        model = Sequential()
        inputShape = (height, width, depth)

        model.add(Conv2D(16, (3, 3), strides=(1, 1), input_shape=inputShape)) 
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        
        model.add(Conv2D(32, (3, 3), strides=(1, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        
        model.add(Conv2D(64, (3, 3), strides=(1, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        model.add(Dropout(0.2))
        
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(classes, activation='softmax'))
        
        print("summary")
        print(model.summary())
        
        return model

In [41]:
EPOCHS = 25
INIT_LR = 0.005
BS = 16

model = Classifier.build(width=2455, height=13, depth=1, classes=8)

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# Callbacks: early stopping and checkpoint
early_stopping = EarlyStopping(monitor='val_accuracy', verbose=1, 
                               patience=10,
                               mode='max',
                               restore_best_weights=True)

filepath = "weights.{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

callbacks_list = [early_stopping, checkpoint]

summary
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 11, 2453, 16)      160       
_________________________________________________________________
activation_3 (Activation)    (None, 11, 2453, 16)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 2452, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 2450, 32)       4640      
_________________________________________________________________
activation_4 (Activation)    (None, 8, 2450, 32)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 2449, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 5, 2447, 

In [42]:
history = model.fit(x_train, y_train, batch_size=BS,
                    validation_data=(x_val, y_val),
                   steps_per_epoch=len(x_train) // BS,
                    callbacks=callbacks_list,
                    epochs=EPOCHS, verbose=1)

Epoch 1/25
399/399 [==============================] - ETA: 0s - loss: 0.7085 - accuracy: 0.1205
Epoch 00001: val_accuracy improved from -inf to 0.12500, saving model to weights.01-0.12.hdf5
399/399 [==============================] - 46s 116ms/step - loss: 0.7085 - accuracy: 0.1205 - val_loss: 0.3768 - val_accuracy: 0.1250
Epoch 2/25
399/399 [==============================] - ETA: 0s - loss: 0.3770 - accuracy: 0.1235
Epoch 00002: val_accuracy did not improve from 0.12500
399/399 [==============================] - 45s 113ms/step - loss: 0.3770 - accuracy: 0.1235 - val_loss: 0.3768 - val_accuracy: 0.1250
Epoch 3/25
399/399 [==============================] - ETA: 0s - loss: 0.3769 - accuracy: 0.1132
Epoch 00003: val_accuracy did not improve from 0.12500
399/399 [==============================] - 45s 113ms/step - loss: 0.3769 - accuracy: 0.1132 - val_loss: 0.3768 - val_accuracy: 0.1250
Epoch 4/25
399/399 [==============================] - ETA: 0s - loss: 0.3769 - accuracy: 0.1184
Epoch 0000

In [0]:
plt.pcolormesh(x_test_prelim[300])